In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as snn

In [2]:
# 导入数据
old_data = pd.read_excel("data_285_313.xlsx",sheet_name="操作变量",header=0) # 读取325个样本的数据，不读取表头
old_data.drop('样本285原始数据',axis= 1,inplace=True)  ##将第一列删除
operation_var = old_data.iloc[1,:]
operation_var = operation_var.to_list()  #获得操作变量的信息
print(len(operation_var))
old_data.drop(0,axis= 0,inplace=True)
old_data.drop(1,axis= 0,inplace=True)
old_data.drop(42,axis=0,inplace=True)

# print(old_data.shape)

old_data = old_data.values
data_285 = old_data[0:40,:]  ## 获得第285号样本的数据
data_313 = old_data[40:80,:] ## 获得第313号样本的数据

df_285 = pd.DataFrame(data_285)
df_313 = pd.DataFrame(data_313)
## 判断有无缺失值
df_285.info()
df_313.info() #均没有缺失值
# print(data_285.shape)


354
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40 entries, 0 to 39
Columns: 354 entries, 0 to 353
dtypes: object(354)
memory usage: 110.8+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40 entries, 0 to 39
Columns: 354 entries, 0 to 353
dtypes: object(354)
memory usage: 110.8+ KB


In [3]:
### 下面根据附件二的要求先对285号样本和313号样本进行数据预处理--第一步

# 首先遍历每一列，找到每一列中0占比较多的列索引，方便问题二建模同，这里将0比例为0.5设为阈值
def find_zero_column(data):
    zero_lst = []
    for i in range(data.shape[1]):
        num = 0
        for j in range(data.shape[0]):
            if data[j,i]==0:
                num = num+1
        if num/data.shape[0]>=0.5:
            zero_lst.append(i)
    return zero_lst
lst1 = find_zero_column(data_285)
lst2 = find_zero_column(data_313)


del_var = list(set(lst1+lst2))  ## 第一步预处理中剔除掉的操作变量，也是问题二中进行模型预测之前需要剔除掉的操作变量


In [13]:
## 第二步根据附件四的信息读取每一个操作变量的所允许的最大值以及最小值
var_constr = pd.read_excel("附件四：354个操作变量信息.xlsx",header=0) 
var_constr.head()

## 读取取值范围所在的列
var_names = var_constr['中文名称']
var_constr = var_constr['取值范围']
var_constrs = var_constr.to_list()  ##获取每个操作变量的取值范围，但此时列表里的元素还不是我们想要的
# print(var_constrs)

## 接下来对上面获得var_constrs 进行切片操作
demo = var_constrs[0]
d = demo.split('-')
var_min = {}  ##存储操作变量最小值的字典
var_max = {}  ##存储操作变量最大值的字典
## 下面对 var_constrs中的元素进行拆分
for i in range(len(var_constrs)): ##遍历var_constrs中的每一个元素
    var = var_constrs[i]
    temp = var.split('-',1) # 设置最大分割次数为1，分隔符为-，注意：有些变量最小值是负数，因此要分类讨论
    # print(temp)
    if temp[0] != '':  ## 如果分割之后的第一个元素不是''说明该数据不是负数，可以被正确存储到var_min中
        var_min[i] = float(temp[0])
        var_max[i] = float(temp[1]) 
    else:
        ## 如果temp[0] =''就意味着最小值是负数
        # 继续分割temp[1]
        var1 = temp[1].split('-',1)
        var_min[i] = -float(var1[0])  ##分割第二次后的第一个元素就是最小值的负数，可以直接添加
        ## 但是对于第二次分割后的第二个元素还要继续细分，如果遇到了括号（），就不能直接添加，所以下面对字符串进行去括号操作
        var2 = var1[1].replace('(','').replace(')','')
        var_max[i] = float(var2)
    
var_min_lst = [j  for i,j in var_min.items()]  ## 354个操作变量的最小取值
var_max_lst = [j  for i,j in var_max.items()]  ## 354个操作变量的最大取值

# var_names.to_csv('操作变量取值.csv')
# new_df = pd.read_csv('操作变量取值.csv')
# new_df['最小值'] = var_min_lst
# new_df['最大值'] = var_max_lst
# new_df.to_csv('操作变量取值.csv',index=False)

In [44]:
# ## 下面对第285号样本以及313号样本进行第二步预处理：筛选操作变量值不在 var_min_lst 以及var_max_lst之间的样本数据重置为0
def second_pre(data):
    minmax_array = np.zeros((data.shape[0],data.shape[1]))  ##初始化一个新的数组
    for i in range(data.shape[1]):  ##遍历每一个操作变量
        for j in range(data.shape[0]): ##遍历每一个时间点
            if (data[j][i] <var_min_lst[i] or data[j][i]>var_max_lst[i]):  ##如果data中的数据不在正常范围内
                minmax_array[j][i] = 0 ##初始该位置的数据为0，方便后续使用
            else:
                minmax_array[j][i] = data[j][i] ##否则不变
    return minmax_array

## 至此，异常值筛选完毕
new_285 = second_pre(data_285)
new_313 = second_pre(data_313)
# print(new_285)

In [45]:
## 根据拉伊达法则对数据进行第三步预处理
def layida(data):
    ## 默认数据的列向量是等精度测量的数据
    mu_data = np.sum(data,axis=0)/data.shape[0]  ##获得数组的平均值
    sigma_data = np.sqrt(np.sum(data**2,axis=0)/(data.shape[0]-1)) ##获得数组的方差值
    new_data = np.zeros((data.shape[0],data.shape[1])) ##初始化新的数组矩阵
    for i in range(data.shape[0]):
        for j in range(data.shape[1]):
            if abs(data[i][j]-mu_data[j])>3*sigma_data[j]: ##如果满足拉伊达法则，就把原数据赋值为0，否则不变
                new_data[i][j] =0
            else:
                new_data[i][j] = data[i][j] 
    return new_data

new1_285 = layida(new_285) 
new1_313 = layida(new_313)
# print(new1_285)
# print(new1_313)

In [46]:
## 统计每一行数据为0的个数，如果该时间点中的数据值为0的个数20，那么将该行数据删除
def count_zero(data,limit):
    zero_num = []
    for i in range(data.shape[0]):
        # zero_num.append(np.sum(np.where(data[i,:]==0,1,0))) ##统计每一行元素为0的个数
        temp = np.sum(np.where(data[i,:]==0,1,0))
        if temp>=limit:
            zero_num.append(i)
    print(zero_num)
    new_data = np.delete(data,zero_num,axis=0)
    return new_data

new2_285 = count_zero(new1_285,20)
new2_313 = count_zero(new1_313,20)
print(new2_285.shape)
print(new2_313.shape)


[]
[18, 19, 25, 27, 37]
(40, 354)
(35, 354)


In [47]:
### 对第285号、313号数据进行最后的处理
last_285 = np.sum(new2_285,axis=0)/new2_285.shape[0]
last_313 = np.sum(new2_313,axis=0)/new2_313.shape[0]

In [48]:
### 将最终处理的数据last_285,last_313添加入data_325.xlsx中
data_325 = pd.read_excel("data_325.xlsx",header=0)
data = data_325.values
data[284,14:] = last_285  # 将整理好的第285号样本数据整理到原数据集中
data[312,14:] = last_313  # 将整理好的第313号样本数据整理到原数据集中
# print(data[0,14])

In [49]:
## 现在对整体数据进行预处理
end_zero_col = find_zero_column(data)  # 将缺失数据较多的操作单位所在的列删除
print(end_zero_col)
## 删除
end_data = np.delete(data,end_zero_col,axis=1)   ## 删除不需要的列
P2_data = np.delete(end_data,9,axis=1)
print(P2_data.shape)


[38, 87, 91, 102, 218, 221, 229, 312]
(325, 359)


In [50]:
y_arr = P2_data[:,7:9]
# print(y_arr)
x_arr = np.delete(P2_data,[7,8],axis=1)
# print(x_arr.shape)

p2_data = np.hstack([x_arr,y_arr]) ##最右边两列是产品性质变量，做预测用，前面的是因变量
import scipy.io as io
io.savemat('p2.mat',{'p2':p2_data})
raw_mater_Var = ['原料硫含量','原料辛烷值','原料饱和烃','原料烯烃','原料芳烃','原料溴值','原料密度','待生吸附-焦碳','待生吸附S','再生吸附-焦炭','再生吸附']
var_name = raw_mater_Var+['产品硫含量','产品辛烷值','产品辛烷值损失'] + operation_var
# print(len(var_name))
var_names = [n for i ,n in enumerate(var_name) if i not in end_zero_col]
var_names.remove('产品辛烷值损失')
var_names.remove('产品辛烷值')
var_names.remove('产品硫含量')
Var_names= var_names+ ['产品硫含量','产品辛烷值']
data = Var_names
filename = 'columns.txt'  # 保存的文件名

with open(filename, 'w') as f:
    for item in data:
        f.write(f"{item}\n")